# Exercise 1 (6 points)

A bacterial population $P$ grows according to the geometric progression

$$P_t = rP_{t-1}$$

Where r is the growth rate. The following population counts $P_1 ,\ldots, P_8$ (in billions) are observed:

In [1]:
import numpy as np

data = np.array( [0.19, 0.36, 0.69, 1.3, 2.5, 4.7, 8.5, 14] )

# (a)
Read chapter 6.6 on Nonlinear Least squares. Use the Gauss-Newton Method to fit the model function $f(t, x_1, x_2) = x_1\!\cdot x_2^t$ to the data. Find estimates for the initial population $P_0=x_1$ and the growth rate $r=x_2$. Implement the Gauss-Newton method yourself (you may use linear algebra functions from `scipy` and `numpy`). 

# (b)
Let $f$ be a vector valued function $f = [ f_1, \ldots, f_m ]^T$. In weighted least squares one aims to minimize the objective function
$$
  \phi(x) = \frac{1}{2} \sum_{i=1}^m W_{ii} ( y_i - f_i(x)) ^2 , \qquad
  W_{ii} = \frac{1}{\sigma_i^2} , 
$$
where $\sigma_i$ is an estimate of the standard deviation in the data point $y_i$. This is equivalent to the standard least squares problem 
$$
\min_x \frac{1}{2} \| Y - F(x) \|_2^2
$$
with $F_i(x) = \frac{1}{\sigma_i} f(x)$  , $Y_i = \frac{1}{\sigma_i} y_i$. Assume that for each data point $y_i$ in the list above, the estimate for the standard deviation is given by
$$
  \sigma_i = 0.05 y_i .
$$ 
Perform a weighted least squares fit to obtain estimates for $P_0$ and $r$. 

Plot the results of (a) and (b), showing the data points and the fitted curve. Compare the residuals
(the values of $y_i - f_i(x)$) obtained in (a) and (b) and discuss the differences between the results of the weighted and the unweighted optimization.


# Exercise 2 (3 points)
A triangle has been measured. The measurements, a vector $x \in \mathbb{R}^6$, are as follows:
$$\begin{array}{c|c|c|c|c|c}
x_1 = \alpha 
& x_2 = \beta
& x_3 = \gamma
& x_4 = a
& x_5 = b
& x_6 = c \\ \hline
67.5^{\large\circ}
& 52^{\large\circ}
& 60^{\large\circ}
& 172 \text{m}
& 146 \text{m}
& 165 \text{m}
\end{array} .
$$
Here $\alpha, \beta, \gamma$ are the angles opposite the sides with length $a$, $b$, $c$, respectively.
The measurements $x$ have errors. We would like to correct them so that the new values $\tilde{x} = x + h$ are consistent quantities of a triangle. The have to satisfy:
$$ \tag{*}
\begin{array}{ccc}
\text{Sum of angles:} 
& \;\;\;\;\; & 
\tilde{x}_1 + \tilde{x}_2 + \tilde{x}_3 = 180^{\large\circ}
\\
\text{Sine theorem:}
&&
\tilde{x}_4 \sin(\tilde{x}_2) - \tilde{x}_5 \sin(\tilde{x}_1) = 0
\\
&&
\tilde{x}_5 \sin(\tilde{x}_3) - \tilde{x}_6 \sin(\tilde{x}_2) = 0 .
\end{array}$$


## (a)
Solve the constrained least squares problem $\min_x \| h \|_2^2$ subject to the constraints given by (*).

Use `scipy.optimize.minimize`.

Hint: Don't forget to work in radians!

Check that for the new values also e.g. the cosine theorem $c^2 = a^2 + b^2 - 2 ab \cos(\gamma)$ holds.

In [12]:
from scipy.optimize import minimize
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

x_deg = np.array([67.5,52,60,172,146,165])
x_rad = np.array([1.178,0.907,1.047,172,146,165])
x_guess = np.array([1,1,1,1,1,1])


def con1(x):
    return(x[0] + x[2] + x[3] - np.pi)


def con2(x):
    return  x[3] * np.sin(x[1]) - x[4] * np.sin(x[0])

def con3(x):
    return x[4] * np.sin(x[2]) - x[5] * np.sin(x[1])


def lst(x,*args):
    h = args[0] - x
    norm = la.norm(h)
    return 0.5 * norm **2

cons = [ {
    'type':'eq',
    'fun': con1
    },
    {
    'type':'ineq',
    'fun': con2
    }, 
    {
    'type':'ineq',
    'fun': con3
    }
]


results = minimize(lst, x_rad, args=(x_guess,),constraints = cons).x



print(results)

# h_guess = results.x


print(results[0]+results[1]+results[2])
print(results[3]*np.sin(results[1])-results[4]*np.sin(results[0]))
test_0 = (results[3]**2 + results[4]**2) - (2*results[3]*results[4]*np.cos(results[2])) - results[5]**2
print(test_0)



[1.04720163 0.99999772 1.0472033  1.04718773 0.99998822 0.99998666]
3.094402641434005
0.015159556252400508
0.04944029273353567


## (b)
You will notice that the corrections will be made mainly to the angles and much less to the lengths of the sides of the triangle. This is because the measurements have not the same absolute errors. While the error in last digit of the sides is about 1, the errors in radians of the angles are about 0.01. Repeat your computation by taking in account with appropriate weighting the difference in measurement errors. Minimize not simply $\| h \|_2^2$ but
$$
  \left\| \begin{bmatrix} 100 h_1 \\ 100 h_2 \\ 100 h_3 \\ h_4 \\ h_5 \\ h_6 \end{bmatrix} \right\|_2^2.$$

In [13]:
def lst_w(x,*args):
    h = args[0] - x
    for i in range(2):
        h[i] *= 100
    norm = la.norm(h)
    return 0.5 * norm **2
    

print(minimize(lst_w, x_guess, args=(x_rad,),constraints = cons))
results = minimize(lst_w, x_guess, args=(x_rad,),constraints = cons).x

test_w = (results[3]**2 + results[4]**2) - (2*results[3]*results[4]*np.cos(results[2])) - results[5]**2
print(test_w)



     fun: 38468.06589619741
     jac: array([-466.77294922,  176.21240234,   -0.86962891, -170.16503906,
       -144.38134766, -164.64501953])
 message: 'Optimization terminated successfully'
    nfev: 79
     nit: 10
    njev: 9
  status: 0
 success: True
       x: array([1.13132592, 0.92462324, 0.17604452, 1.83422221, 1.61819775,
       0.3549697 ])
0.012413193267118539
